In [1]:
# 1분 16초
from utils import build_model_and_tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from quant import quant, dequant, quant_unpack, dequant_unpack

model_name = 'facebook/opt-6.7b'
device = "cuda:1"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
testenc = tokenizer("\n\n".join(dataset["text"]), return_tensors="pt")

bits = 4
gs = 128
scale, zero, qs = quant(bits, gs, model)
q_x = dequant(scale, zero, qs, gs, bits)

/home/jwjeong/anaconda3/envs/autogptq/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]


In [3]:
for key in scale:
    print(f'shape of scale = {scale[key].size()}')
    print(f'shape of zeros = {zero[key].size()}')
    print(f'shape of qs = {qs[key].size()}')

shape of scale = torch.Size([4096, 32])
shape of zeros = torch.Size([4096, 32])
shape of qs = torch.Size([512, 4096])
shape of scale = torch.Size([4096, 32])
shape of zeros = torch.Size([4096, 32])
shape of qs = torch.Size([512, 4096])
shape of scale = torch.Size([4096, 32])
shape of zeros = torch.Size([4096, 32])
shape of qs = torch.Size([512, 4096])
shape of scale = torch.Size([4096, 32])
shape of zeros = torch.Size([4096, 32])
shape of qs = torch.Size([512, 4096])
shape of scale = torch.Size([16384, 32])
shape of zeros = torch.Size([16384, 32])
shape of qs = torch.Size([2048, 4096])
shape of scale = torch.Size([4096, 128])
shape of zeros = torch.Size([4096, 128])
shape of qs = torch.Size([512, 16384])
shape of scale = torch.Size([4096, 32])
shape of zeros = torch.Size([4096, 32])
shape of qs = torch.Size([512, 4096])
shape of scale = torch.Size([4096, 32])
shape of zeros = torch.Size([4096, 32])
shape of qs = torch.Size([512, 4096])
shape of scale = torch.Size([4096, 32])
shape of z

: 

#### pack, unpack은 계산상의 차이는 없음
#### 다만, unpack으로는 low-bit value 분석에 사용하면 될듯!